In [10]:
import pandas as pd
import scipy.io as sio
import numpy as np
import os
import re
import time

from numpy.lib import recfunctions as rfn

In [13]:
def convert_matlab_to_pd_and_clean(file_name):

    # Import Matlab file to Pandas
    matlab_filename = file_name

    mat = sio.loadmat(matlab_filename, squeeze_me=True)
    mat_records = np.rec.array(mat['result'])
    mat_records = rfn.drop_fields(mat_records, "debug")

    df = pd.DataFrame.from_records(mat_records)

    # Cleanup color names
    df['cs'] = df['cs'].str.replace(".*\\\\", "")
    df['cs'] = df['cs'].str.replace("\\.jpg", "")

    df['csi'] = df['csi'].str.replace(".*\\\\", "")
    df['csi'] = df['csi'].str.replace("\\.jpg", "")
    
    # Convert response frequencies to pd Series
    i_respSeries = df['i_respPerTrialPerSec'].apply(pd.Series).fillna(value=0)
    t_respSeries = df['t_respPerTrialPerSec'].apply(pd.Series).fillna(value=0)

    df = pd.concat([df, t_respSeries], axis=1)
    df = pd.concat([df, i_respSeries], axis=1)
   
    return df


In [14]:
dir_path = "."

matlab_file_pattern = re.compile("(\d+-)+\d+.*\.mat")

now = time.strftime("%Y%m%d_%H%M%S")
writer = pd.ExcelWriter(now + '.xlsx')

for f in os.listdir(dir_path):
  if matlab_file_pattern.match(f):
    df = convert_matlab_to_pd_and_clean(f)
    df.to_excel(writer, f)
    
writer.save()